In [1]:
from ultralytics import YOLO
import torch
import os
import shutil
from pathlib import Path
import numpy as np
import random
import zipfile
import pandas as pd
import cv2

In [2]:
# set random seed
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# Load dataset from kaggle

In [3]:
# this cell is for running in Kaggle
# load kaggle.json from google drive
# !gdown <item id>

In [4]:
# check if the kaggle.json file exists
from sys import platform

# Windows
# C:\Users\<Windows-username>\.kaggle\kaggle.json
if platform == "win32":
    print("Windows")
    kaggle_json_path = os.path.join(os.path.expanduser("~"), ".kaggle", "kaggle.json")
    if os.path.exists(kaggle_json_path):
        print("kaggle.json exists")
    else:
        print("kaggle.json does not exist")
        shutil.copy("./kaggle.json", kaggle_json_path)
        print("kaggle.json is copied to the directory")

# Linux
# ~/.kaggle/kaggle.json
elif platform == "linux" or platform == "linux2":
    print("Linux")
    kaggle_json_path = os.path.join(os.path.expanduser("~"), ".kaggle", "kaggle.json")
    if os.path.exists(kaggle_json_path):
        print("kaggle.json exists")
    else:
        print("kaggle.json does not exist")
        os.makedirs(os.path.join(os.path.expanduser("~"), ".kaggle"), exist_ok=True)
        shutil.copy("./kaggle.json", kaggle_json_path)
        print("kaggle.json is copied to the directory")

# macOS
# ~/.kaggle/kaggle.json
elif platform == "darwin":
    print("macOS")
    kaggle_json_path = os.path.join(os.path.expanduser("~"), ".kaggle", "kaggle.json")
    if os.path.exists(kaggle_json_path):
        print("kaggle.json exists")
    else:
        print("kaggle.json does not exist")
        os.makedirs(os.path.join(os.path.expanduser("~"), ".kaggle"), exist_ok=True)
        shutil.copy("./kaggle.json", kaggle_json_path)
        print("kaggle.json is copied to the directory")

Linux
kaggle.json exists


In [5]:
%%script False
# download and unzip the dataset
if(os.path.exists("./rice-image-dataset-for-object-detection")):
    print("The dataset is already downloaded")
else:
    !kaggle datasets download -d alikhalilit98/rice-image-dataset-for-object-detection
    with zipfile.ZipFile("rice-image-dataset-for-object-detection.zip", 'r') as zip_ref:
        zip_ref.extractall("./rice-image-dataset-for-rice-germ-detection")

Couldn't find program: 'False'


In [5]:
raw_dataset_path = './rice-image-dataset-for-rice-germ-detection'
dataset_path = './rice-germ-cls-dataset'
Path(dataset_path).mkdir(parents=True, exist_ok=True)
Path(f'{dataset_path}/train/Good').mkdir(parents=True, exist_ok=True)
Path(f'{dataset_path}/train/Moderate').mkdir(parents=True, exist_ok=True)
Path(f'{dataset_path}/train/Bad').mkdir(parents=True, exist_ok=True)
Path(f'{dataset_path}/val/Good').mkdir(parents=True, exist_ok=True)
Path(f'{dataset_path}/val/Moderate').mkdir(parents=True, exist_ok=True)
Path(f'{dataset_path}/val/Bad').mkdir(parents=True, exist_ok=True)

In [6]:
%%script False
!gdown 1Gk73cVdNwL4PYF37XRBKa-lybbdE0k7O -O ./rice-germ-cls-dataset/rice-germ-1050labels.zip

Downloading...
From: https://drive.google.com/uc?id=1Gk73cVdNwL4PYF37XRBKa-lybbdE0k7O
To: /home/andre/Desktop/ComputerVision_STA_Project/rice-germ-cls-dataset/rice-germ-1050labels.zip
100%|█████████████████████████████████████████| 206k/206k [00:00<00:00, 971kB/s]


In [7]:
%%script False
with zipfile.ZipFile(f"{dataset_path}/rice-germ-1050labels.zip", 'r') as zip_ref:
    zip_ref.extractall(path=dataset_path)

# Splitting dataset

In [8]:
df = pd.read_csv(f"{dataset_path}/rice-germ-1050labels.csv")
df.sort_values(by=['image_name'], inplace=True)
df[:10]

,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height
0,Bad,155,93,50,57,Jasmine (1).jpg,250,250
1,Bad,156,106,49,64,Jasmine (10).jpg,250,250
2,Moderate,130,124,60,63,Jasmine (100).jpg,250,250
392,Good,64,62,48,44,Jasmine (1006).jpg,250,250
3,Moderate,112,49,61,59,Jasmine (101).jpg,250,250
393,Good,93,158,51,43,Jasmine (1010).jpg,250,250
4,Moderate,74,151,51,56,Jasmine (102).jpg,250,250
5,Good,157,96,46,52,Jasmine (103).jpg,250,250
6,Bad,88,150,61,60,Jasmine (104).jpg,250,250
7,Bad,152,94,60,61,Jasmine (105).jpg,250,250


In [9]:
from sklearn.model_selection import train_test_split

raw_labels_path = f"{dataset_path}/rice-germ-1050labels"
raw_images_path = f"{raw_dataset_path}/Rice_Image_Dataset_OD/Jasmine"

data_lists = os.listdir(raw_labels_path)
data_lists = [data_list.split(".")[0] for data_list in data_lists]
print(data_lists[:10])

train, val = train_test_split(data_lists, test_size=0.1, random_state=random_seed, stratify=df["label_name"])
print(f"train: {len(train)}, val: {len(val)}")

['Jasmine (293)', 'Jasmine (3287)', 'Jasmine (31)', 'Jasmine (1299)', 'Jasmine (3412)', 'Jasmine (6657)', 'Jasmine (3239)', 'Jasmine (5391)', 'Jasmine (3092)', 'Jasmine (143)']
train: 945, val: 105


In [10]:
%%script False
for x in train:
    img_path = f"{raw_images_path}/{x}.jpg"
    label_path = f"{raw_labels_path}/{x}.txt"
    with open(label_path, "r") as f:
        label = f.read(1) # 0: Good, 1: Moderate, 2: Bad
    if label == "0":
        shutil.copy(img_path, f"{dataset_path}/train/Good")
    elif label == "1":
        shutil.copy(img_path, f"{dataset_path}/train/Moderate")
    elif label == "2":
        shutil.copy(img_path, f"{dataset_path}/train/Bad")

for x in val:
    img_path = f"{raw_images_path}/{x}.jpg"
    label_path = f"{raw_labels_path}/{x}.txt"
    with open(label_path, "r") as f:
        label = f.read(1) # 0: Good, 1: Moderate, 2: Bad
    if label == "0":
        shutil.copy(img_path, f"{dataset_path}/val/Good")
    elif label == "1":
        shutil.copy(img_path, f"{dataset_path}/val/Moderate")
    elif label == "2":
        shutil.copy(img_path, f"{dataset_path}/val/Bad")

# Create yaml file

In [ ]:
import yaml

full_path = os.path.abspath(dataset_path)

yaml_data = dict(
    path = full_path,
    train='train',
    val='val',
    nc=3,
    names=['Good', 'Moderate', 'Bad']
)

with open(f"{dataset_path}/data.yaml", "w") as f:
    yaml.dump(yaml_data, f, default_flow_style=False, sort_keys=False)

# Train model

In [1]:
from ultralytics import YOLO
import torch
import os
import shutil
from pathlib import Path
import numpy as np
import random
import zipfile
import pandas as pd
import cv2

random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

In [104]:
# a = unlabeled_data_picker.pick(10)
# print(a)

Unlabeled data: 13930
['Jasmine (4123)' 'Jasmine (6539)' 'Jasmine (4839)' 'Jasmine (13975)' 'Jasmine (7419)' 'Jasmine (11923)' 'Jasmine (8068)' 'Jasmine (12823)' 'Jasmine (1758)' 'Jasmine (12641)']


In [2]:
# training parameters
epochs = 500
batch = 32
imgsz = 128
dataset_path = './rice-germ-cls-dataset'

# Teacher Model

In [ ]:
model = YOLO("yolov8x-cls.pt")
config = {
    "data": dataset_path,
    "epochs": epochs,
    "batch": batch,
    "imgsz": imgsz,
    "device": "0",
    "project": "rice-germ-clssification",
    "name": "run3",
    "seed": random_seed,
    "cache": True,
    "workers": 24,
    "patience": epochs,
    "save": False,
    "exist_ok": False,
}
trained_model = model.train(**config)
# torch.cuda.empty_cache()

# Iterative KD

In [1]:
from ultralytics import YOLO
import torch
import os
import shutil
from pathlib import Path
import numpy as np
import random
import zipfile
import pandas as pd
import cv2

inject_new_data = True
epochs = 150

random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

dataset_path = './rice-germ-cls-dataset'
raw_dataset_path = './rice-image-dataset-for-rice-germ-detection'
raw_images_path = f"{raw_dataset_path}/Rice_Image_Dataset_OD/Jasmine"

In [2]:
a1 = ["train", "val"]
a2 = ["Good", "Moderate", "Bad"]
label_data_set = set()
# tt = os.listdir(f"{dataset_path}/train/Good")
# tt = [t.split(".")[0] for t in tt]
for i in a1:
    for j in a2:
        tt = os.listdir(f"{dataset_path}/{i}/{j}")
        tt = [t.split(".")[0] for t in tt]
        print(f"{i}/{j}: {len(tt)}")
        label_data_set = label_data_set.union(set(tt))

alldata = os.listdir(raw_images_path)
alldata = [data.split(".")[0] for data in alldata]
alldata_set = set(alldata)

print(f"Total data: {len(alldata_set)}, Labeled data: {len(label_data_set)}")

unlabeled_data = alldata_set - label_data_set

print(f"Unlabeled data: {len(unlabeled_data)}")

class unlabel_picker:
    def __init__(self, data):
        self.unlabeled_data_set = data
    
    def pick(self, n):
        selected = np.random.choice(list(self.unlabeled_data_set), n, replace=False)
        self.unlabeled_data_set = self.unlabeled_data_set - set(selected)
        print(f"Unlabeled data: {len(self.unlabeled_data_set)}")
        return selected

    def __len__(self):
        return len(self.unlabeled_data_set)

unlabeled_data_picker = unlabel_picker(unlabeled_data)
        

train/Good: 376
train/Moderate: 485
train/Bad: 434
val/Good: 32
val/Moderate: 42
val/Bad: 31
Total data: 15000, Labeled data: 1400
Unlabeled data: 13600


In [3]:
run_path = "./rice-germ-clssification"
model_path = "run4"
model_path = os.path.join(run_path, model_path)
model_path_pt = os.path.join(model_path, "weights", "last.pt")

myModel = YOLO(model_path_pt)

if inject_new_data:
    N = 50
    for i in unlabeled_data_picker.pick(N):
        a = myModel.predict(f"{raw_images_path}/{i}.jpg")
        label = a[0].names[a[0].probs.top1]
        shutil.copy(f"{raw_images_path}/{i}.jpg", f"{dataset_path}/train/{label}")

batch = 128
imgsz = 128
config = {
    "data": dataset_path,
    "epochs": epochs,
    "batch": batch,
    "imgsz": imgsz,
    "device": "0",
    "project": "rice-germ-clssification",
    "name": "run4",
    "seed": random_seed,
    "cache": True,
    "workers": 24,
    "patience": epochs,
    "save": False,
    "exist_ok": True,
}

myModel.train(**config)

Unlabeled data: 13550

image 1/1 /home/andre/Desktop/ComputerVision_STA_Project/rice-image-dataset-for-rice-germ-detection/Rice_Image_Dataset_OD/Jasmine/Jasmine (9489).jpg: 128x128 Moderate 0.80, Bad 0.20, Good 0.00, 2.0ms
Speed: 1.5ms preprocess, 2.0ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)

image 1/1 /home/andre/Desktop/ComputerVision_STA_Project/rice-image-dataset-for-rice-germ-detection/Rice_Image_Dataset_OD/Jasmine/Jasmine (1124).jpg: 128x128 Moderate 0.95, Good 0.03, Bad 0.02, 1.9ms
Speed: 1.1ms preprocess, 1.9ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)

image 1/1 /home/andre/Desktop/ComputerVision_STA_Project/rice-image-dataset-for-rice-germ-detection/Rice_Image_Dataset_OD/Jasmine/Jasmine (3913).jpg: 128x128 Good 0.71, Moderate 0.28, Bad 0.00, 1.9ms
Speed: 0.8ms preprocess, 1.9ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)

image 1/1 /home/andre/Desktop/ComputerVision_STA_Project/rice-image-dataset-for-rice-g

train: Scanning /home/andre/Desktop/ComputerVision_STA_Project/rice-germ-cls-dataset/train... 1345 images, 0 corrupt: 100%|██████████| 1345/1345 [00:00<00:00, 16221.28it/s]

train: New cache created: /home/andre/Desktop/ComputerVision_STA_Project/rice-germ-cls-dataset/train.cache



val: Scanning /home/andre/Desktop/ComputerVision_STA_Project/rice-germ-cls-dataset/val... 105 images, 0 corrupt: 100%|██████████| 105/105 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 50 weight(decay=0.0), 51 weight(decay=0.001), 51 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 128 train, 128 val
Using 24 dataloader workers
Logging results to rice-germ-clssification/run4
Starting training for 150 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

                   all       0.79          1



150 epochs completed in 0.039 hours.
Optimizer stripped from rice-germ-clssification/run4/weights/last.pt, 112.6MB
Optimizer stripped from rice-germ-clssification/run4/weights/best.pt, 112.5MB

Validating rice-germ-clssification/run4/weights/best.pt...
Ultralytics YOLOv8.1.37 🚀 Python-3.11.8 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16071MiB)
YOLOv8x-cls summary (fused): 133 layers, 56127043 parameters, 0 gradients, 153.8 GFLOPs
train: /home/andre/Desktop/ComputerVision_STA_Project/rice-germ-cls-dataset/train... found 1345 images in 3 classes ✅ 
val: /home/andre/Desktop/ComputerVision_STA_Project/rice-germ-cls-dataset/val... found 105 images in 3 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00, 59.77it/s]


                   all      0.876          1
Speed: 0.0ms preprocess, 0.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to rice-germ-clssification/run4
Results saved to rice-germ-clssification/run4


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x73139c6aba50>
curves: []
curves_results: []
fitness: 0.9380952417850494
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8761904835700989, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9380952417850494}
save_dir: PosixPath('rice-germ-clssification/run4')
speed: {'preprocess': 0.009629839942568824, 'inference': 0.14105070204961867, 'loss': 3.1789143880208336e-05, 'postprocess': 2.724783761160714e-05}
task: 'classify'
top1: 0.8761904835700989
top5: 1.0

In [4]:
assert False

AssertionError: 

In [5]:
myModel.val()

Ultralytics YOLOv8.1.37 🚀 Python-3.11.8 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16071MiB)
YOLOv8x-cls summary (fused): 133 layers, 56127043 parameters, 0 gradients, 153.8 GFLOPs
train: /home/andre/Desktop/ComputerVision_STA_Project/rice-germ-cls-dataset/train... found 1345 images in 3 classes ✅ 
val: /home/andre/Desktop/ComputerVision_STA_Project/rice-germ-cls-dataset/val... found 105 images in 3 classes ✅ 
test: None...


val: Scanning /home/andre/Desktop/ComputerVision_STA_Project/rice-germ-cls-dataset/val... 105 images, 0 corrupt: 100%|██████████| 105/105 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00,  4.11it/s]


                   all      0.876          1
Speed: 0.0ms preprocess, 1.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to rice-germ-clssification/run4


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7312985f2110>
curves: []
curves_results: []
fitness: 0.9380952417850494
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8761904835700989, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9380952417850494}
save_dir: PosixPath('rice-germ-clssification/run4')
speed: {'preprocess': 0.010865075247628348, 'inference': 1.02079709370931, 'loss': 5.222502208891369e-05, 'postprocess': 0.00010899135044642856}
task: 'classify'
top1: 0.8761904835700989
top5: 1.0

# Export model as onnx

In [5]:
run_path = "./rice-germ-clssification"
model_path = "run4" # change this to the model path you want to use
model_path = os.path.join(run_path, model_path)
model_path_pt = os.path.join(model_path, "weights", "best.pt")

In [6]:
myModel = YOLO(model_path_pt)
myModel.val()

Ultralytics YOLOv8.1.37 🚀 Python-3.11.8 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080, 16071MiB)
YOLOv8x-cls summary (fused): 133 layers, 56127043 parameters, 0 gradients, 153.8 GFLOPs
train: /home/andre/Desktop/ComputerVision_STA_Project/rice-germ-cls-dataset/train... found 1345 images in 3 classes ✅ 
val: /home/andre/Desktop/ComputerVision_STA_Project/rice-germ-cls-dataset/val... found 105 images in 3 classes ✅ 
test: None...


val: Scanning /home/andre/Desktop/ComputerVision_STA_Project/rice-germ-cls-dataset/val... 105 images, 0 corrupt: 100%|██████████| 105/105 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 7/7 [00:00<00:00, 39.25it/s]


                   all      0.876          1
Speed: 0.0ms preprocess, 1.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val2


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f528e466010>
curves: []
curves_results: []
fitness: 0.9380952417850494
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8761904835700989, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9380952417850494}
save_dir: PosixPath('runs/classify/val2')
speed: {'preprocess': 0.02768834431966146, 'inference': 1.3545513153076172, 'loss': 0.00021117074148995535, 'postprocess': 0.0002702077229817708}
task: 'classify'
top1: 0.8761904835700989
top5: 1.0

In [7]:
to_save_model = YOLO(model_path_pt)
to_save_model.export(format="onnx")
model_path_onnx = os.path.join(model_path, "weights", "best.onnx")

Ultralytics YOLOv8.1.37 🚀 Python-3.11.8 torch-2.1.1+cu121 CPU (13th Gen Intel Core(TM) i7-13700K)
YOLOv8x-cls summary (fused): 133 layers, 56127043 parameters, 0 gradients, 153.8 GFLOPs

PyTorch: starting from 'rice-germ-clssification/run4/weights/best.pt' with input shape (1, 3, 128, 128) BCHW and output shape(s) (1, 3) (107.3 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 1.1s, saved as 'rice-germ-clssification/run4/weights/best.onnx' (214.1 MB)

Export complete (2.5s)
Results saved to /home/andre/Desktop/ComputerVision_STA_Project/rice-germ-clssification/run4/weights
Predict:         yolo predict task=classify model=rice-germ-clssification/run4/weights/best.onnx imgsz=128  
Validate:        yolo val task=classify model=rice-germ-clssification/run4/weights/best.onnx imgsz=128 data=./rice-germ-cls-dataset  
Visualize:       https://netron.app


In [ ]:
import datetime
from pathlib import Path

to_save_path = "./saved_model"
Path(to_save_path).mkdir(parents=True, exist_ok=True)

time = datetime.datetime.now().strftime("%d%m%y%H%M")

# zip run17 folder into name seg-model-<ddmmyyhhmm>.zip
zip_name = f" rice-germ-cls-model-{time}"

shutil.make_archive(f"{to_save_path}/{zip_name}_imgsz{imgsz}", 'zip', model_path)

KeyboardInterrupt: 